In [1]:
import pandas as pd
import xarray as xr

from tqdm.notebook import tqdm

# Generate discharge hindcast data for BEST-Siberian marine modelling

## Part 1

Aims of this notebook are to:

**1. Fill RADR data to full model time period** 
* Read in monthly RADR simulated discharge data for all catchment outlets to the Arctic Ocean (1984-2018), and convert to a dataframe for easier manipulation.
* Fill RADR data forward or backwards to cover the model time period of interest (1980-2019, incl.).

## Read in RADR modelled discharge

Decide to drop the netcdf format and convert to a pandas dataframe for easier manipulation in the rest of the notebook.

Then reindex to the full period 1980-2019 (inclusive). Fill NaNs at the start and end of the period using monthly means for the first or last 5 years, respectively.

In [2]:
# Read in RADR monthly time series of mean daily discharge
radr_monthly_path = r"/home/jovyan/shared/pan_arctic_data/discharge/RADR_v1_monthly.nc"
ds = xr.open_dataset(radr_monthly_path)
ds

<xarray.Dataset>
Dimensions:    (time: 420, reach: 10658)
Coordinates:
  * time       (time) datetime64[ns] 1984-01-31 1984-02-29 ... 2018-12-31
  * reach      (reach) float64 2.5e+07 2.5e+07 2.5e+07 ... 8.601e+07 8.601e+07
Data variables:
    discharge  (time, reach) float32 ...

In [3]:
# Convert to a dataframe, wide format
df = ds.to_dataframe().unstack('reach')
df.columns = [int(i) for i in df.columns.get_level_values(1)]
df.head()

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
time,,,,,,,,,,,,,,,,,,,,,
1984-01-31,0.127290,0.257097,0.009097,0.087806,0.006613,0.005258,0.011161,0.003839,0.036806,0.069677,...,0.005484,0.004516,0.005032,0.002742,0.001000,0.273290,0.001548,0.000258,0.001387,0.001000
1984-02-29,0.060207,0.146897,0.005483,0.055586,0.003448,0.002966,0.006138,0.002103,0.021207,0.037897,...,0.002241,0.002897,0.003069,0.001828,0.000034,0.156586,0.001000,0.000000,0.001000,0.001000
1984-03-31,0.031717,0.092551,0.004021,0.037828,0.002159,0.001957,0.003688,0.001168,0.016884,0.023016,...,0.001032,0.002000,0.002032,0.001000,0.000000,0.121452,0.000968,0.000000,0.000548,0.001000
1984-04-30,0.017378,0.062136,0.002923,0.027059,0.001948,0.001016,0.002467,0.001028,0.012234,0.014986,...,0.000500,0.002000,0.002000,0.001000,0.000000,0.097733,0.000000,0.000000,0.000000,0.001000
1984-05-31,7.727692,9.951077,0.034122,0.578305,1.627073,0.009455,2.919397,1.767511,0.565087,2.944690,...,0.274286,1.286728,3.015106,0.824171,0.545592,0.768520,0.210349,0.435876,0.353809,0.749504


## Fill start years (1980-1983)

In [4]:
# Reindex to include the earlier years that we also want to include
idx = pd.date_range(start='1980-01', end='2019-01', freq='M')
df = df.reindex(idx)
df.loc['1983-10': '1984-03']

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
1983-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-01-31,0.127290,0.257097,0.009097,0.087806,0.006613,0.005258,0.011161,0.003839,0.036806,0.069677,...,0.005484,0.004516,0.005032,0.002742,0.001000,0.273290,0.001548,0.000258,0.001387,0.001
1984-02-29,0.060207,0.146897,0.005483,0.055586,0.003448,0.002966,0.006138,0.002103,0.021207,0.037897,...,0.002241,0.002897,0.003069,0.001828,0.000034,0.156586,0.001000,0.000000,0.001000,0.001
1984-03-31,0.031717,0.092551,0.004021,0.037828,0.002159,0.001957,0.003688,0.001168,0.016884,0.023016,...,0.001032,0.002000,0.002032,0.001000,0.000000,0.121452,0.000968,0.000000,0.000548,0.001


In [5]:
# Work out monthly means for a period of interest (for now, just the full period)
st_yr = 1984
end_yr = 1988

# Add month column to df
df['Month'] = df.index.month

# Calculate monthly means
monthly_mean_df = df.loc["%s-01" % st_yr: "%s-12" % end_yr].groupby('Month').mean()
monthly_mean_df

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
Month,,,,,,,,,,,,,,,,,,,,,
1,0.413871,0.820594,0.010594,0.101600,0.048916,0.007155,0.152890,0.054852,0.081677,0.199974,...,0.005839,0.004600,0.004658,0.002729,0.000890,0.326748,0.001445,0.000161,0.001245,0.001000
2,0.278020,0.626647,0.006567,0.079199,0.058765,0.007019,0.115152,0.057078,0.054251,0.212095,...,0.002329,0.002935,0.002936,0.001809,0.000007,0.185897,0.001000,0.000000,0.001000,0.001000
3,0.315723,0.590573,0.004875,0.048182,0.039955,0.004355,0.143316,0.051397,0.050263,0.191426,...,0.001187,0.002071,0.002026,0.001006,0.000000,0.133135,0.000865,0.000000,0.000419,0.001000
4,0.204567,0.439349,0.004329,0.033002,0.023134,0.002644,0.101797,0.030544,0.050069,0.119323,...,0.000575,0.002000,0.002000,0.001000,0.000002,0.106013,0.000036,0.000002,0.000002,0.001000
5,4.514033,6.885776,0.055937,0.647493,1.373359,0.142193,1.684965,1.102751,0.756961,2.894806,...,0.254963,0.662341,1.285055,0.364558,0.224215,0.556527,0.068412,0.189670,0.155152,0.317712
6,18.020884,61.036026,1.784181,8.367931,10.096654,3.374911,7.677880,7.207571,15.050660,17.495531,...,0.697215,2.221441,2.483534,1.058206,0.420369,1.150312,0.188314,0.335631,0.331340,0.694017
7,4.244188,20.129440,3.434175,3.426013,1.961866,1.492444,1.800872,1.552267,18.702236,4.943917,...,0.356788,1.297134,1.586937,0.616160,0.267017,0.665578,0.107373,0.203519,0.177406,0.455780
8,4.886766,13.362019,2.313039,1.952374,1.994945,0.495398,1.897073,1.477773,10.981636,4.832197,...,1.489136,1.746937,2.437218,0.922952,0.430264,2.245301,0.172089,0.288673,0.284402,0.626648
9,6.088940,15.759263,0.487759,2.071197,2.478096,0.577332,2.242153,1.843654,3.772047,5.251565,...,1.572538,1.430937,1.791653,0.722885,0.333128,2.651695,0.141707,0.217950,0.212836,0.478155


In [6]:
# Fill NaNs with monthly means

# fill_df = df.iloc[:, 0:3].copy()
fill_df = df

fill_df['Month'] = fill_df.index.month

# First, reindex df using just the month of the year, so have same indices as
# monthly_mean_df. Makes it easy to map the value to use for NaN filling
fill_df = fill_df.set_index('Month')

# Loop through columns and fill NaNs using monthly means
for col in tqdm(fill_df.columns):
    fill_df[col] = fill_df[col].fillna(monthly_mean_df[col])

# Back to the original datetime index
fill_df.set_index(idx, inplace=True)

fill_df

  0%|          | 0/10658 [00:00<?, ?it/s]

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
1980-01-31,0.413871,0.820594,0.010594,0.101600,0.048916,0.007155,0.152890,0.054852,0.081677,0.199974,...,0.005839,0.004600,0.004658,0.002729,0.000890,0.326748,0.001445,0.000161,0.001245,0.001000
1980-02-29,0.278020,0.626647,0.006567,0.079199,0.058765,0.007019,0.115152,0.057078,0.054251,0.212095,...,0.002329,0.002935,0.002936,0.001809,0.000007,0.185897,0.001000,0.000000,0.001000,0.001000
1980-03-31,0.315723,0.590573,0.004875,0.048182,0.039955,0.004355,0.143316,0.051397,0.050263,0.191426,...,0.001187,0.002071,0.002026,0.001006,0.000000,0.133135,0.000865,0.000000,0.000419,0.001000
1980-04-30,0.204567,0.439349,0.004329,0.033002,0.023134,0.002644,0.101797,0.030544,0.050069,0.119323,...,0.000575,0.002000,0.002000,0.001000,0.000002,0.106013,0.000036,0.000002,0.000002,0.001000
1980-05-31,4.514033,6.885776,0.055937,0.647493,1.373359,0.142193,1.684965,1.102751,0.756961,2.894806,...,0.254963,0.662341,1.285055,0.364558,0.224215,0.556527,0.068412,0.189670,0.155152,0.317712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-31,39.152313,130.873276,23.305212,19.288410,16.017300,4.916871,20.105921,15.554638,78.606720,41.341919,...,2.122537,1.929311,2.625944,1.126598,0.646529,1.595380,0.349283,0.368732,0.477788,0.717840
2018-09-30,11.358512,51.126183,4.329095,5.037320,2.040549,2.652604,9.046653,2.401566,18.287642,6.110734,...,0.188655,0.369890,0.305538,0.145210,0.035321,0.453854,0.022677,0.017805,0.025378,0.101135
2018-10-31,8.298501,42.962948,0.324985,0.738942,0.118241,0.609840,8.941764,0.389098,2.283957,0.613033,...,0.007783,0.049472,0.042899,0.022471,0.005289,0.017791,0.006128,0.003472,0.006368,0.010634
2018-11-30,9.168729,37.837696,0.099223,0.279555,0.060684,0.113644,8.220037,0.397612,0.679518,0.322445,...,0.002400,0.015233,0.013633,0.007533,0.001633,0.004800,0.002600,0.001167,0.002567,0.003000


## Fill forward to 2020

In [16]:
# Now add in 2019
idx = pd.date_range(start='1980-01', end='2020-01', freq='M')
fill_df = fill_df.reindex(idx)
fill_df.loc['2018-09': '2019-02']

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
2018-09-30,11.358512,51.126183,4.329095,5.037320,2.040549,2.652604,9.046653,2.401566,18.287642,6.110734,...,0.188655,0.369890,0.305538,0.145210,0.035321,0.453854,0.022677,0.017805,0.025378,0.101135
2018-10-31,8.298501,42.962948,0.324985,0.738942,0.118241,0.609840,8.941764,0.389098,2.283957,0.613033,...,0.007783,0.049472,0.042899,0.022471,0.005289,0.017791,0.006128,0.003472,0.006368,0.010634
2018-11-30,9.168729,37.837696,0.099223,0.279555,0.060684,0.113644,8.220037,0.397612,0.679518,0.322445,...,0.002400,0.015233,0.013633,0.007533,0.001633,0.004800,0.002600,0.001167,0.002567,0.003000
2018-12-31,11.102421,25.765388,0.038387,0.134484,0.038935,0.022871,6.396968,0.138613,0.155032,0.166161,...,0.001323,0.006774,0.006097,0.003516,0.001000,0.002323,0.001355,0.000452,0.001258,0.001226
2019-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Work out monthly means for a period of interest
st_yr = 2014
end_yr = 2018

# Calculate monthly means
monthly_mean_df = df.loc["%s-01" % st_yr: "%s-12" % end_yr].groupby('Month').mean()
monthly_mean_df

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
Month,,,,,,,,,,,,,,,,,,,,,
1,12.941754,12.426554,0.014465,0.086381,0.010942,0.008484,4.234103,0.019587,0.047503,0.065716,...,0.009413,0.003019,0.002871,0.001716,0.000335,0.125129,0.000800,0.000000,0.000800,0.000800
2,13.264210,8.048677,0.007126,0.052759,0.006230,0.003806,3.688496,0.012353,0.022877,0.036139,...,0.003350,0.002059,0.001967,0.001047,0.000000,0.052943,0.000800,0.000000,0.000744,0.000800
3,11.190213,5.748310,0.005381,0.035085,0.003601,0.002162,2.784845,0.004020,0.013298,0.019885,...,0.001542,0.001600,0.001600,0.000800,0.000000,0.023290,0.000310,0.000000,0.000000,0.000800
4,8.182920,3.471107,0.003231,0.024487,0.002296,0.001316,1.874876,0.001864,0.008767,0.012428,...,0.000962,0.001602,0.001602,0.000802,0.000003,0.010368,0.000003,0.000003,0.000003,0.000802
5,5.737019,3.239577,0.007023,0.052351,0.015259,0.004347,1.178961,0.089747,0.070482,0.055133,...,0.000796,0.006801,0.044906,0.003202,0.015524,0.005056,0.000075,0.001976,0.002036,0.002757
6,7.657146,16.106339,3.916366,2.165707,1.642887,2.262202,2.313538,3.123791,24.134857,1.155872,...,2.092002,3.442569,5.842332,2.571578,1.406528,1.354198,0.638437,1.276669,1.157917,1.956762
7,28.613062,92.355881,21.871717,13.285915,12.458604,6.085263,14.116137,12.632565,90.110901,25.785990,...,2.042458,2.237881,2.827060,1.612056,0.739471,3.685202,0.565064,0.423521,0.518723,0.976147
8,21.085857,92.354813,16.413288,12.975287,8.371559,4.170100,13.274123,11.907501,61.358551,19.910418,...,2.222395,1.799249,2.398038,1.107335,0.622606,2.595284,0.340799,0.378970,0.467496,0.693224
9,12.018847,49.889977,3.071530,3.938724,1.689519,1.981234,9.954576,4.351581,15.834708,4.421781,...,0.794947,0.436335,0.438504,0.212688,0.090679,3.524111,0.065163,0.039261,0.073994,0.112599


In [17]:
# Fill NaNs with monthly means

fill_df['Month'] = fill_df.index.month

# First, reindex df using just the month of the year, so have same indices as
# monthly_mean_df. Makes it easy to map the value to use for NaN filling
fill_df = fill_df.set_index('Month')

# Loop through columns and fill NaNs using monthly means
for col in tqdm(fill_df.columns):
    fill_df[col] = fill_df[col].fillna(monthly_mean_df[col])

# Back to the original datetime index
fill_df.set_index(idx, inplace=True)

fill_df

  0%|          | 0/10658 [00:00<?, ?it/s]

,25000001,25000003,25000008,25000009,25000010,25000012,25000014,25000015,25000018,25000020,...,86007262,86007263,86007264,86007266,86007268,86007270,86007271,86007272,86007273,86007275
1980-01-31,0.413871,0.820594,0.010594,0.101600,0.048916,0.007155,0.152890,0.054852,0.081677,0.199974,...,0.005839,0.004600,0.004658,0.002729,0.000890,0.326748,0.001445,0.000161,0.001245,0.001000
1980-02-29,0.278020,0.626647,0.006567,0.079199,0.058765,0.007019,0.115152,0.057078,0.054251,0.212095,...,0.002329,0.002935,0.002936,0.001809,0.000007,0.185897,0.001000,0.000000,0.001000,0.001000
1980-03-31,0.315723,0.590573,0.004875,0.048182,0.039955,0.004355,0.143316,0.051397,0.050263,0.191426,...,0.001187,0.002071,0.002026,0.001006,0.000000,0.133135,0.000865,0.000000,0.000419,0.001000
1980-04-30,0.204567,0.439349,0.004329,0.033002,0.023134,0.002644,0.101797,0.030544,0.050069,0.119323,...,0.000575,0.002000,0.002000,0.001000,0.000002,0.106013,0.000036,0.000002,0.000002,0.001000
1980-05-31,4.514033,6.885776,0.055937,0.647493,1.373359,0.142193,1.684965,1.102751,0.756961,2.894806,...,0.254963,0.662341,1.285055,0.364558,0.224215,0.556527,0.068412,0.189670,0.155152,0.317712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-31,21.085857,92.354813,16.413288,12.975287,8.371559,4.170100,13.274123,11.907501,61.358551,19.910418,...,2.222395,1.799249,2.398038,1.107335,0.622606,2.595284,0.340799,0.378970,0.467496,0.693224
2019-09-30,12.018847,49.889977,3.071530,3.938724,1.689519,1.981234,9.954576,4.351581,15.834708,4.421781,...,0.794947,0.436335,0.438504,0.212688,0.090679,3.524111,0.065163,0.039261,0.073994,0.112599
2019-10-31,11.650717,43.543720,0.438476,1.163739,0.402185,0.574673,9.047188,1.615698,3.257380,1.435619,...,0.400989,0.054006,0.052837,0.027239,0.007424,1.892262,0.010856,0.004307,0.010770,0.010686
2019-11-30,12.285815,30.737495,0.208407,0.582640,0.231246,0.206203,6.753450,0.326240,1.060203,0.516882,...,0.128053,0.016080,0.015400,0.008500,0.002060,0.772353,0.003847,0.001273,0.003693,0.003027


## Save

In [19]:
# Round to 3 decimal places and save
fill_df = fill_df.round(3)

radr_filled_fpath = r"/home/jovyan/shared/pan_arctic_data/discharge/RADR_v1_monthly_filled_1980-2019.csv"
fill_df.to_csv(radr_filled_fpath, index_label="Date")